In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200,200,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(layers.Flatten()) 
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 99, 99, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 23, 23, 128)      

In [ ]:
# mounting the files

from google.colab import drive
drive.mount('/content/gdrive')
!unzip /content/gdrive/MyDrive/Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/training_set/cats/cat.1428.jpg  
  inflating: Dataset/training_set/cats/cat.1437.jpg  
  inflating: Dataset/training_set/cats/cat.1429.jpg  
  inflating: Dataset/training_set/cats/cat.1436.jpg  
  inflating: Dataset/training_set/cats/cat.144.jpg  
  inflating: Dataset/training_set/cats/cat.142.jpg  
  inflating: Dataset/training_set/cats/cat.1658.jpg  
  inflating: Dataset/training_set/cats/cat.1461.jpg  
  inflating: Dataset/training_set/cats/cat.1444.jpg  
  inflating: Dataset/training_set/cats/cat.1440.jpg  
  inflating: Dataset/training_set/dogs/dog.544.jpg  
  inflating: Dataset/training_set/dogs/dog.64.jpg  
  inflating: Dataset/training_set/cats/cat.1431.jpg  
  inflating: Dataset/test_set/cats/cat.4578.jpg  
  inflating: Dataset/training_set/cats/cat.185.jpg  
  inflating: Dataset/training_set/cats/cat.1434.jpg  
  inflating: Dataset/training_set/cats/cat.1560.jpg  
  inflating: Dataset/training_set/cats/ca

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)


In [ ]:
training_data = datagen.flow_from_directory('/content/Dataset/training_set/',class_mode = 'categorical', batch_size = 532, target_size=(200,200))
testing_data = datagen.flow_from_directory('/content/Dataset/test_set/',class_mode = 'categorical', batch_size = 203, target_size=(200,200))

Found 5319 images belonging to 2 classes.
Found 2032 images belonging to 2 classes.


In [ ]:
#
              metrics=['accuracy'])

In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt,metrics=['accuracy'])

In [ ]:
history = model.fit(training_data,validation_data=testing_data, epochs=10)

Epoch 1/10
10/10 [==============================] - 24s 2s/step - loss: 0.5287 - accuracy: 0.7500 - val_loss: 0.7545 - val_accuracy: 0.5015
Epoch 2/10
10/10 [==============================] - 23s 2s/step - loss: 0.5306 - accuracy: 0.7516 - val_loss: 0.6871 - val_accuracy: 0.5015
Epoch 3/10
10/10 [==============================] - 23s 2s/step - loss: 0.5391 - accuracy: 0.7502 - val_loss: 0.6854 - val_accuracy: 0.5015
Epoch 4/10
10/10 [==============================] - 23s 2s/step - loss: 0.5614 - accuracy: 0.7543 - val_loss: 0.7293 - val_accuracy: 0.5015
Epoch 5/10
10/10 [==============================] - 25s 3s/step - loss: 0.5300 - accuracy: 0.7525 - val_loss: 0.7095 - val_accuracy: 0.5015
Epoch 6/10
10/10 [==============================] - 24s 2s/step - loss: 0.5295 - accuracy: 0.7524 - val_loss: 0.7134 - val_accuracy: 0.5015
Epoch 7/10
10/10 [==============================] - 24s 2s/step - loss: 0.5328 - accuracy: 0.7489 - val_loss: 0.7365 - val_accuracy: 0.5015
Epoch 8/10
10/10 [==

In [ ]:
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='red', label='train')
pyplot.plot(history.history['val_loss'], color='magenta', label='test')
pyplot.show()


In [ ]:
#accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'], color='brown', label='train')
pyplot.plot(history.history['val_accuracy'], color='lime', label='test')
pyplot.show()

In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

#**TENSOR FLOW LITE converstion**

Export the saved model
  

In [ ]:

export_dir = "LITE_model"
tf.saved_model.save(model, export_dir)

In [ ]:
import pathlib


 Convert the keras model to a tflite model

In [ ]:

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()



Save the tflite model

In [ ]:

tflite_model_file = pathlib.Path('tflite_model.tflite')
tflite_model_file.write_bytes(tflite_model)


Optimizing the tflite model

In [ ]:


converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
litemodel_catdog = converter.convert()


Total number of Bytes in the saved model 


In [ ]:


tflite_model_file = pathlib.Path('litemodel_catdog.tflite')
tflite_model_file.write_bytes(litemodel_catdog)